# 量子とは何か？

In [11]:
# This code is to create the interactive figure
# このコードはインタラクティブな図形を作成するためのものです
from bokeh.events import ButtonClick
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CustomJS, Button, Slider, DataRange1d, LabelSet, RadioButtonGroup, Div
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.io import output_notebook, show, curdoc
import IPython

「量子物理学」は広く使用されている用語ですが、あまり理解されていません。「量子物理学」は「古典」物理学の理論では説明できない実験室での小さなものの振る舞いを説明するために、最初に使用された数学モデルです。 量子論はこの振る舞いを説明し、私たちの世界のより完全な全体像を与えてくれます。 そして以前は説明できなかったこの振る舞いを使用して、不可能と思われていた特定の計算を実行できることに気付きました。 これを量子コンピューティングと呼びます。

量子コンピューティングは、量子物理学に踏み出す最適な方法です。 量子物理学の核となる概念を最も単純な形に抽出し、物理世界の複雑さを取り除きます。 それでは、いくつかの奇妙な量子現象を発見（そして説明！）して、「量子」が何であるかを味わっていきましょう。

## 古典的確率の復習

量子現象を理解するために、私たちは最初に「古典的な」確率を思い出す必要があります。 ここでの「古典的な」とは、量子より前、つまり学校で習うはずの正規確率ツリーを意味します。 この分野に既に精通している場合は、すぐに読み進めることができます。 それほど精通していない場合でも、心配しないでください。可能な限り最も単純な確率の問題をいくつか取り上げていきます。

### 確率ツリー

もしかしたら学校で習った確率ツリーを覚えているかもしれません。 アイデアは単純です。図面を使用して、起こりうるすべての不測の事態をマッピングし、発生する確率を計算していきます。

例えばコインを持っているとしましょう。まず、それをヘッドの状態に置きます。 次に、この公正なコインを投げてそれを見ると、50％の確率で再びヘッドが表示され、50％の確率でテイルが表示されます。 これを次のように確率ツリーにプロットできます。

![](images/whatis/whatis1.png)

各枝の終わりに結果を描画し、枝での各発生の確率を描画します。 同様に、テイルの状態で開始してコインを投げた場合、50％の確率でヘッドが表示され、50％の確率でテイルが表示されます。

![](images/whatis/whatis2.png)

試してみることでこの動作をテストできます。 物理的にコインを取り出し、何度も実行し、それぞれの結果を記録することができます。 最終的に、結果の約50％がヘッドで50％がテイルであることがわかります。 信頼できる結果を得るには、約500〜1000回のトスで十分です。

### 実験 #1: シングルコイントス

下の`Toss Coin`ボタンを押してコイントスをシミュレートし、結果を保存することで、コイントス実験をシミュレートできます。 初期状態を「ヘッド」または「テイル」に変更したり、コインの数（`No. of Coins`）スライダーを増やしたりして、多くの結果をすばやく得ることができます。結果を破棄し、やり直すためには`Reset`を押してください。

In [13]:
data_labels = ["Heads", "Tails"]
initial_counts = [0]*2
initial_labels = ["0.00%"]*2

source = ColumnDataSource(data=dict(x=data_labels, top=initial_counts, top_percent=initial_labels))

plot = figure(
              plot_height=500, 
              plot_width=500,
              sizing_mode="scale_width",
              tools="reset",
              x_range=data_labels,
              y_range=DataRange1d(start=0, range_padding = 20, range_padding_units='absolute'),
              y_axis_label="Counts")
plot.vbar(top='top',x='x',width=0.5,source=source, color="#BE95FF")
labels = LabelSet(x='x', y='top', text='top_percent', level='glyph',
        x_offset=-20, y_offset=12, source=source, render_mode='canvas')
plot.add_layout(labels)



# ボタンとスライダーを定義する
tossbtn = Button(label="Toss Coin")
repeat_slider = Slider(title="No. of Coins", start=1, end=50, step=1, value=0)
radio_label = Div(text="Initial State:")
init_state_radio = RadioButtonGroup(name="Initial State", labels=data_labels, active=0)
resetbtn = Button(label="Reset")
results_label = Div(text="Results:")
resultsHTML = Div(text="", style={'overflow-y': 'auto', 'max-height': '250px'})

# コールバックを定義する
toss_callback = CustomJS(args=dict(source=source, p=plot, 
                                   s=repeat_slider, 
                                   resultsHTML=resultsHTML, 
                                   results_label=results_label), 
                code="""
        for (var i = 0; i <= s.value; ++i){
            const result = Math.floor(Math.random() * 2);
            source.data.top[result] += 1;
            resultsHTML.text += [' H ', ' T '][result]
        }
        const total_tosses = source.data.top[0] + source.data.top[1];
        results_label.text = "Results (" + total_tosses + " tosses):";
        
        for (var i = 0; i<2; ++i){
            const frac = source.data.top[i] / source.data.top.reduce((a, b) => a + b, 0);
            source.data.top_percent[i] = (frac*100).toFixed(2) + "%";
        }
        if (Math.max(...source.data.top) > 22) {
            p.y_range.range_padding_units = 'percent';
            p.y_range.range_padding = 1;
        } else {
            p.y_range.range_padding_units = 'absolute';
            p.y_range.range_padding = 30 - Math.max(...source.data.top);
        };
        source.change.emit();
    """)

slider_callback = CustomJS(args=dict(tossbtn=tossbtn), 
                  code="""
    const repeats = cb_obj.value;
    if (repeats == 1) {
        tossbtn.label = "Toss Coin";
    }
    else {
        tossbtn.label = "Toss " + repeats + " Coins";
    };
    """)

reset_callback = CustomJS(args=dict(source=source, resultsHTML=resultsHTML, results_label=results_label), code="""
    source.data.top = [0,0];
    source.data.top_percent = ['0.00%', '0.00%'];
    source.change.emit();
    resultsHTML.text = "";
    results_label.text = "Results:";
    """)

# コールバックをボタン/スライダーにリンクする
tossbtn.js_on_event(ButtonClick, toss_callback)
repeat_slider.js_on_change('value', slider_callback)
resetbtn.js_on_event(ButtonClick, reset_callback)

# レイアウトの作成
control_panel = column(tossbtn, 
                       repeat_slider, 
                       radio_label, 
                       init_state_radio, 
                       resetbtn, 
                       results_label, 
                       resultsHTML)

layout = row(plot, control_panel)

# HTMLを出力
html_repr = file_html(layout, CDN)
IPython.display.HTML(html_repr)

### さらに続けると

確率ツリーモデルが実験結果を正しく予測しているようです。 さらに進んで、確率ツリーを連鎖させて、一連のイベントの結果を予測することができます。 たとえば、ヘッド状態から始めて、コインを投げてから、_コインをもう一度投げる_ としましょう。何が表示されますか？ ツリーを使ってみてみましょう。

![](images/whatis/whatis3.png)


イベントの各組み合わせの確率を計算するために、枝に沿って乗算していきます。

![](images/whatis/whatis4.gif)

次に、結果を合計して、各結果の確率を計算します。

![](images/whatis/whatis5.gif)

2回のトス後にヘッドとなる確率は50％であり、テイルとなる確率も50％であることがわかります。

### 実験 #2: ダブルコイントス

実験1と同様に、`Toss Coin Twice` ボタンを押して2つのコイントスをシミュレートし、最終結果を保存することで、コイントス実験をシミュレートできます。 初期状態や、`No. of Coins` を変更することで、多くの結果をすばやく得ることができます。結果を破棄し、やり直すためには、`Reset`を押してください。

In [15]:
data_labels = ["Heads", "Tails"]
initial_counts = [0]*2
initial_labels = ["0.00%"]*2

source = ColumnDataSource(data=dict(x=data_labels, top=initial_counts, top_percent=initial_labels))

plot = figure(
              plot_height=500, 
              plot_width=500,
              sizing_mode="scale_width",
              tools="reset",
              x_range=data_labels,
              y_range=DataRange1d(start=0, range_padding = 20, range_padding_units='absolute'),
              y_axis_label="Counts")
plot.vbar(top='top',x='x',width=0.5,source=source, color="#BE95FF")
labels = LabelSet(x='x', y='top', text='top_percent', level='glyph',
        x_offset=-20, y_offset=12, source=source, render_mode='canvas')
plot.add_layout(labels)



# ボタンとスライダーを定義する
tossbtn = Button(label="Toss Coin Twice")
repeat_slider = Slider(title="No. of Coins", start=1, end=50, step=1, value=0)
radio_label = Div(text="Initial State:")
init_state_radio = RadioButtonGroup(name="Initial State", labels=data_labels, active=0)
resetbtn = Button(label="Reset")
results_label = Div(text="Results:")
resultsHTML = Div(text="", style={'overflow-y': 'auto', 'max-height': '250px'})

# コールバックを定義する
toss_callback = CustomJS(args=dict(source=source, p=plot, 
                                   s=repeat_slider, 
                                   resultsHTML=resultsHTML, 
                                   results_label=results_label), 
                code="""
        for (var i = 0; i <= s.value; ++i){
            const result = Math.floor(Math.random() * 2);
            source.data.top[result] += 1;
            resultsHTML.text += [' H ', ' T '][result]
        }
        const total_tosses = source.data.top[0] + source.data.top[1];
        results_label.text = "Results (" + total_tosses + " Double Tosses):";
        
        for (var i = 0; i<2; ++i){
            const frac = source.data.top[i] / source.data.top.reduce((a, b) => a + b, 0);
            source.data.top_percent[i] = (frac*100).toFixed(2) + "%";
        }
        if (Math.max(...source.data.top) > 22) {
            p.y_range.range_padding_units = 'percent';
            p.y_range.range_padding = 1;
        } else {
            p.y_range.range_padding_units = 'absolute';
            p.y_range.range_padding = 30 - Math.max(...source.data.top);
        };
        source.change.emit();
    """)

slider_callback = CustomJS(args=dict(tossbtn=tossbtn), 
                  code="""
    const repeats = cb_obj.value;
    if (repeats == 1) {
        tossbtn.label = "Toss Coin Twice";
    }
    else {
        tossbtn.label = "Toss " + repeats + " Coins Twice";
    };
    """)

reset_callback = CustomJS(args=dict(source=source, resultsHTML=resultsHTML, results_label=results_label), code="""
    source.data.top = [0,0];
    source.data.top_percent = ['0.00%', '0.00%'];
    source.change.emit();
    resultsHTML.text = "";
    results_label.text = "Results:";
    """)

# コールバックをボタン/スライダーにリンクする
tossbtn.js_on_event(ButtonClick, toss_callback)
repeat_slider.js_on_change('value', slider_callback)
resetbtn.js_on_event(ButtonClick, reset_callback)

# レイアウトの作成
control_panel = column(tossbtn, 
                       repeat_slider, 
                       radio_label, 
                       init_state_radio, 
                       resetbtn, 
                       results_label, 
                       resultsHTML)

layout = row(plot, control_panel)

# HTMLを出力
html_repr = file_html(layout, CDN)
IPython.display.HTML(html_repr)

十分なトスを行うと、ヘッドまたはテイルを測定するのは等しい確率となり期待通りです。

## 量子コイン

古典的なコインの完全な説明ができたところで次は、量子「コイン」を紹介します。 量子コインは「キュービット」と呼ばれています。

キュービットは操作が非常に難しいため、ラボでしか遊べないものです。 長年の科学的および技術的進歩により、今日のキュービットが作成されています。量子コンピューティングを通して学習することで、物理的な複雑さを無視することができます。またキュービットを測定すると2つの状態のうち1つになることを覚えておいてください。 ヘッドとテイルの2つの状態の代わりに、キュービットの2つの状態を0と1と呼びます。

### 実験 #3: 量子コイントス

量子コインを試して、どのように動作するかを見てみましょう。 量子トスを行い、コインの状態を測定して記録します。 これは、上記のセクションの古典的なコイントスと同じです。

In [16]:
data_labels = ["0", "1"]
initial_counts = [0]*2
initial_labels = ["0.00%"]*2

source = ColumnDataSource(data=dict(x=data_labels, top=initial_counts, top_percent=initial_labels))

plot = figure(
              plot_height=500, 
              plot_width=500,
              sizing_mode="scale_width",
              tools="reset",
              x_range=data_labels,
              y_range=DataRange1d(start=0, range_padding = 20, range_padding_units='absolute'),
              y_axis_label="Counts")
plot.vbar(top='top',x='x',width=0.5,source=source, color="#BE95FF")
labels = LabelSet(x='x', y='top', text='top_percent', level='glyph',
        x_offset=-20, y_offset=12, source=source, render_mode='canvas')
plot.add_layout(labels)



# ボタンとスライダーを定義する
tossbtn = Button(label="Toss Quantum Coin")
repeat_slider = Slider(title="No. of Coins", start=1, end=50, step=1, value=0)
radio_label = Div(text="Initial State:")
init_state_radio = RadioButtonGroup(name="Initial State", labels=data_labels, active=0)
resetbtn = Button(label="Reset")
results_label = Div(text="Results:")
resultsHTML = Div(text="", style={'overflow-y': 'auto', 'max-height': '250px'})

# コールバックを定義する
toss_callback = CustomJS(args=dict(source=source, p=plot, 
                                   s=repeat_slider, 
                                   resultsHTML=resultsHTML, 
                                   results_label=results_label), 
                code="""
        for (var i = 0; i <= s.value; ++i){
            const result = Math.floor(Math.random() * 2);
            source.data.top[result] += 1;
            resultsHTML.text += [' 0 ', ' 1 '][result]
        }
        const total_tosses = source.data.top[0] + source.data.top[1];
        results_label.text = "Results (" + total_tosses + " Quantum Tosses):";
        
        for (var i = 0; i<2; ++i){
            const frac = source.data.top[i] / source.data.top.reduce((a, b) => a + b, 0);
            source.data.top_percent[i] = (frac*100).toFixed(2) + "%";
        }
        if (Math.max(...source.data.top) > 22) {
            p.y_range.range_padding_units = 'percent';
            p.y_range.range_padding = 1;
        } else {
            p.y_range.range_padding_units = 'absolute';
            p.y_range.range_padding = 30 - Math.max(...source.data.top);
        };
        source.change.emit();
    """)

slider_callback = CustomJS(args=dict(tossbtn=tossbtn), 
                  code="""
    const repeats = cb_obj.value;
    if (repeats == 1) {
        tossbtn.label = "Toss Quantum Coin";
    }
    else {
        tossbtn.label = "Toss " + repeats + " Quantum Coins";
    };
    """)

reset_callback = CustomJS(args=dict(source=source, resultsHTML=resultsHTML, results_label=results_label), code="""
    source.data.top = [0,0];
    source.data.top_percent = ['0.00%', '0.00%'];
    source.change.emit();
    resultsHTML.text = "";
    results_label.text = "Results:";
    """)

# コールバックをボタン/スライダーにリンクする
tossbtn.js_on_event(ButtonClick, toss_callback)
repeat_slider.js_on_change('value', slider_callback)
resetbtn.js_on_event(ButtonClick, reset_callback)

# レイアウトの作成
control_panel = column(tossbtn, 
                       repeat_slider, 
                       radio_label, 
                       init_state_radio, 
                       resetbtn, 
                       results_label, 
                       resultsHTML)

layout = row(plot, control_panel)

# HTMLを出力
html_repr = file_html(layout, CDN)
IPython.display.HTML(html_repr)

確率ツリーを使用して、量子コインについて説明します。 これは、0の状態から、コイントスで0または1を測定する確率が50-50になるように見えます。古典的なコインで行ったように、これをツリーにプロットしてみましょう。

![](images/whatis/whatis6.png)

In [17]:
data_labels = ["0", "1"]
initial_counts = [0]*2
initial_labels = ["0.00%"]*2

source = ColumnDataSource(data=dict(x=data_labels, top=initial_counts, top_percent=initial_labels))

plot = figure(
              plot_height=500, 
              plot_width=500,
              sizing_mode="scale_width",
              tools="reset",
              x_range=data_labels,
              y_range=DataRange1d(start=0, range_padding = 20, range_padding_units='absolute'),
              y_axis_label="Counts")
plot.vbar(top='top',x='x',width=0.5,source=source, color="#BE95FF")
labels = LabelSet(x='x', y='top', text='top_percent', level='glyph',
        x_offset=-20, y_offset=12, source=source, render_mode='canvas')
plot.add_layout(labels)



# ボタンとスライダーを定義する
tossbtn = Button(label="Toss Quantum Coin")
repeat_slider = Slider(title="No. of Coins", start=1, end=50, step=1, value=0)
radio_label = Div(text="Initial State:")
init_state_radio = RadioButtonGroup(name="Initial State", labels=data_labels, active=1)
resetbtn = Button(label="Reset")
results_label = Div(text="Results:")
resultsHTML = Div(text="", style={'overflow-y': 'auto', 'max-height': '250px'})

# コールバックを定義する
toss_callback = CustomJS(args=dict(source=source, p=plot, 
                                   s=repeat_slider, 
                                   resultsHTML=resultsHTML, 
                                   results_label=results_label), 
                code="""
        for (var i = 0; i <= s.value; ++i){
            const result = Math.floor(Math.random() * 2);
            source.data.top[result] += 1;
            resultsHTML.text += [' 0 ', ' 1 '][result]
        }
        const total_tosses = source.data.top[0] + source.data.top[1];
        results_label.text = "Results (" + total_tosses + " Quantum Tosses):";
        
        for (var i = 0; i<2; ++i){
            const frac = source.data.top[i] / source.data.top.reduce((a, b) => a + b, 0);
            source.data.top_percent[i] = (frac*100).toFixed(2) + "%";
        }
        if (Math.max(...source.data.top) > 22) {
            p.y_range.range_padding_units = 'percent';
            p.y_range.range_padding = 1;
        } else {
            p.y_range.range_padding_units = 'absolute';
            p.y_range.range_padding = 30 - Math.max(...source.data.top);
        };
        source.change.emit();
    """)

slider_callback = CustomJS(args=dict(tossbtn=tossbtn), 
                  code="""
    const repeats = cb_obj.value;
    if (repeats == 1) {
        tossbtn.label = "Toss Quantum Coin";
    }
    else {
        tossbtn.label = "Toss " + repeats + " Quantum Coins";
    };
    """)

reset_callback = CustomJS(args=dict(source=source, resultsHTML=resultsHTML, results_label=results_label), code="""
    source.data.top = [0,0];
    source.data.top_percent = ['0.00%', '0.00%'];
    source.change.emit();
    resultsHTML.text = "";
    results_label.text = "Results:";
    """)

# コールバックをボタン/スライダーにリンクする
tossbtn.js_on_event(ButtonClick, toss_callback)
repeat_slider.js_on_change('value', slider_callback)
resetbtn.js_on_event(ButtonClick, reset_callback)

# レイアウトの作成
control_panel = column(tossbtn, 
                       repeat_slider, 
                       radio_label, 
                       init_state_radio, 
                       resetbtn, 
                       results_label, 
                       resultsHTML)

layout = row(plot, control_panel)

# HTMLを出力
html_repr = file_html(layout, CDN)
IPython.display.HTML(html_repr)

同様に、1の状態から、コイントスにより0または1を測定する確率は50-50になります。確率ツリーは次のようになります。

![](images/whatis/whatis7.png)

### 実験 #4: ダブル量子コイントス

これで、量子コインの動作を予測するモデルができました。 優れた科学者のように、新しいシナリオでそれをテストし、正しいか確認したいと思います。 以前と同じようにダブルコイントスを試してみましょう。 古典的なコインと同じように、量子コインのモデルでは、どの状態で開始するかに関係なく、0または1を測定する確率が50-50であると予測しています。

![](images/whatis/whatis8.png)

さあ、やってみましょう！ 量子コインを2回投げます。

In [18]:
data_labels = ["0", "1"]
initial_counts = [0]*2
initial_labels = ["0.00%"]*2

source = ColumnDataSource(data=dict(x=data_labels, top=initial_counts, top_percent=initial_labels))

plot = figure(
              plot_height=500, 
              plot_width=500,
              sizing_mode="scale_width",
              tools="reset",
              x_range=data_labels,
              y_range=DataRange1d(start=0, range_padding = 20, range_padding_units='absolute'),
              y_axis_label="Counts")
plot.vbar(top='top',x='x',width=0.5,source=source, color="#BE95FF")
labels = LabelSet(x='x', y='top', text='top_percent', level='glyph',
        x_offset=-20, y_offset=12, source=source, render_mode='canvas')
plot.add_layout(labels)



# ボタンとスライダーを定義する
tossbtn = Button(label="Toss Quantum Coin Twice")
repeat_slider = Slider(title="No. of Coins", start=1, end=50, step=1, value=0)
radio_label = Div(text="Initial State:")
init_state_radio = RadioButtonGroup(name="Initial State", labels=data_labels, active=0)
resetbtn = Button(label="Reset")
results_label = Div(text="Results:")
resultsHTML = Div(text="", style={'overflow-y': 'auto', 'max-height': '250px'})

# コールバックを定義する
toss_callback = CustomJS(args=dict(source=source, p=plot, 
                                   s=repeat_slider, 
                                   resultsHTML=resultsHTML, 
                                   results_label=results_label,
                                init_state_radio=init_state_radio), 
                code="""
        for (var i = 0; i <= s.value; ++i){
            const result = init_state_radio.active;
            source.data.top[result] += 1;
            resultsHTML.text += [' 0 ', ' 1 '][result]
        }
        const total_tosses = source.data.top[0] + source.data.top[1];
        results_label.text = "Results (" + total_tosses + " Double Quantum Tosses):";
        
        for (var i = 0; i<2; ++i){
            const frac = source.data.top[i] / source.data.top.reduce((a, b) => a + b, 0);
            source.data.top_percent[i] = (frac*100).toFixed(2) + "%";
        }
        if (Math.max(...source.data.top) > 22) {
            p.y_range.range_padding_units = 'percent';
            p.y_range.range_padding = 1;
        } else {
            p.y_range.range_padding_units = 'absolute';
            p.y_range.range_padding = 30 - Math.max(...source.data.top);
        };
        source.change.emit();
    """)

slider_callback = CustomJS(args=dict(tossbtn=tossbtn), 
                  code="""
    const repeats = cb_obj.value;
    if (repeats == 1) {
        tossbtn.label = "Toss Quantum Coin Twice";
    }
    else {
        tossbtn.label = "Toss " + repeats + " Quantum Coins Twice";
    };
    """)

reset_callback = CustomJS(args=dict(source=source, resultsHTML=resultsHTML, results_label=results_label), code="""
    source.data.top = [0,0];
    source.data.top_percent = ['0.00%', '0.00%'];
    source.change.emit();
    resultsHTML.text = "";
    results_label.text = "Results:";
    """)

# コールバックをボタン/スライダーにリンクする
tossbtn.js_on_event(ButtonClick, toss_callback)
repeat_slider.js_on_change('value', slider_callback)
resetbtn.js_on_event(ButtonClick, reset_callback)

# レイアウトの作成
control_panel = column(tossbtn, 
                       repeat_slider, 
                       radio_label, 
                       init_state_radio, 
                       resetbtn, 
                       results_label, 
                       resultsHTML)

layout = row(plot, control_panel)

# HTMLを出力
html_repr = file_html(layout, CDN)
IPython.display.HTML(html_repr)

うーん…これは予想外の結果です。 初期状態を1に設定するとどうなるか見てみましょう。

In [19]:
data_labels = ["0", "1"]
initial_counts = [0]*2
initial_labels = ["0.00%"]*2

source = ColumnDataSource(data=dict(x=data_labels, top=initial_counts, top_percent=initial_labels))

plot = figure(
              plot_height=500, 
              plot_width=500,
              sizing_mode="scale_width",
              tools="reset",
              x_range=data_labels,
              y_range=DataRange1d(start=0, range_padding = 20, range_padding_units='absolute'),
              y_axis_label="Counts")
plot.vbar(top='top',x='x',width=0.5,source=source, color="#BE95FF")
labels = LabelSet(x='x', y='top', text='top_percent', level='glyph',
        x_offset=-20, y_offset=12, source=source, render_mode='canvas')
plot.add_layout(labels)



# ボタンとスライダーを定義する
tossbtn = Button(label="Toss Quantum Coin Twice")
repeat_slider = Slider(title="No. of Coins", start=1, end=50, step=1, value=0)
radio_label = Div(text="Initial State:")
init_state_radio = RadioButtonGroup(name="Initial State", labels=data_labels, active=1)
resetbtn = Button(label="Reset")
results_label = Div(text="Results:")
resultsHTML = Div(text="", style={'overflow-y': 'auto', 'max-height': '250px'})

# コールバックを定義する
toss_callback = CustomJS(args=dict(source=source, p=plot, 
                                   s=repeat_slider, 
                                   resultsHTML=resultsHTML, 
                                   results_label=results_label,
                                init_state_radio=init_state_radio), 
                code="""
        for (var i = 0; i <= s.value; ++i){
            const result = init_state_radio.active;
            source.data.top[result] += 1;
            resultsHTML.text += [' 0 ', ' 1 '][result]
        }
        const total_tosses = source.data.top[0] + source.data.top[1];
        results_label.text = "Results (" + total_tosses + " Quantum Tosses)";
        
        for (var i = 0; i<2; ++i){
            const frac = source.data.top[i] / source.data.top.reduce((a, b) => a + b, 0);
            source.data.top_percent[i] = (frac*100).toFixed(2) + "%";
        }
        if (Math.max(...source.data.top) > 22) {
            p.y_range.range_padding_units = 'percent';
            p.y_range.range_padding = 1;
        } else {
            p.y_range.range_padding_units = 'absolute';
            p.y_range.range_padding = 30 - Math.max(...source.data.top);
        };
        source.change.emit();
    """)

slider_callback = CustomJS(args=dict(tossbtn=tossbtn), 
                  code="""
    const repeats = cb_obj.value;
    if (repeats == 1) {
        tossbtn.label = "Toss Quantum Coin Twice";
    }
    else {
        tossbtn.label = "Toss " + repeats + " Quantum Coins Twice";
    };
    """)

reset_callback = CustomJS(args=dict(source=source, resultsHTML=resultsHTML, results_label=results_label), code="""
    source.data.top = [0,0];
    source.data.top_percent = ['0.00%', '0.00%'];
    source.change.emit();
    resultsHTML.text = "";
    results_label.text = "Results:";
    """)

# コールバックをボタン/スライダーにリンクする
tossbtn.js_on_event(ButtonClick, toss_callback)
repeat_slider.js_on_change('value', slider_callback)
resetbtn.js_on_event(ButtonClick, reset_callback)

# レイアウトの作成
control_panel = column(tossbtn, 
                       repeat_slider, 
                       radio_label, 
                       init_state_radio, 
                       resetbtn, 
                       results_label, 
                       resultsHTML)

layout = row(plot, control_panel)

# HTMLを出力
html_repr = file_html(layout, CDN)
IPython.display.HTML(html_repr)

私たちの予測とはまったく一致しません！ このモデルでは説明できません！ これは、物理学者が20世紀初頭に遭遇したのと同じ問題です。 答えを探すことで、量子物理学の発展につながりました。量子コイントスを説明するためにその答えを使ってみましょう。

## 量子モデル

要するに、量子論は負の数の確率論です。

これは何を意味するのでしょうか？ 意味のない負の確率を持つことはできません。 これに対応するために、_振幅_ と呼ばれる新しい量を使用し、代わりにこれらをツリーにプロットします。 負の確率を持つことはできず、すべての確率を合計して1にする必要があるという事実のため、数学的なトリックを使用します。振幅を2乗して確率を計算します。

例を見てみましょう。 単一量子コイントスの振幅ツリーは次のようになります。

![](images/whatis/whatis9.png)

状態0から開始して、量子コイントスが両方の結果に等しい振幅を割り当てることがわかります。 これらの振幅を2乗すると、0または1を測定する正しい確率が得られます（50-50の確率）。 振幅が $\sqrt{\tfrac{1}{2}}$ であることをどうやって知りましたか？ こうすると正しい答えを得られるからです！

状態1から始めると、振幅ツリーは異なります。

![](images/whatis/whatis10.png)

1の結果の振幅に最初の負の数が現れています。 振幅を二乗して確率を計算すると、この負の符号が消え（負の数×負の値は正であることを思い出してください）、上記を測定すると50-50の確率となります。 興味深い結果は、これらを連鎖させるとおきます。

## ダブル量子コイントスの説明

古典的な確率と同様に、分岐に沿って振幅を乗算して、各結果の振幅を計算します。

![](images/whatis/whatis11.gif)

そして、各結果を測定する確率を計算するために、これらの振幅を合計し、次にそれらを二乗します。

![](images/whatis/whatis12.gif)

状態1のコイン（キュービット）を見つける振幅が互いに打ち消し合っていることがわかります。これを干渉と呼びます。 初期状態が1のときにこのモデルが機能することを確かめてください。

## 量子コンピューティングとは何か？

干渉効果は有用であることがわかりました。 量子コイントスなどの操作を組み合わせて、より効率的なアルゴリズムを構築できます。 これらのアルゴリズムは、干渉効果を使用して、間違った答えをすばやくキャンセルし、正しい答えを測定する可能性を高くすることができます。 これが量子コンピューティングの背後にある考え方です。